In [1]:
import numpy as np

In [2]:
files = np.load('./parameters.npz')
W1 = files.get('W1')
W2 = files.get('W2')
W3 = files.get('W3')
b1 = files.get('b1')
b2 = files.get('b2')
b3 = files.get('b3')
files.close()

x = np.array([[1],[0],[1]])
y = np.array([[0],[0],[1]])

W = [W1, W2, W3]
b = [b1, b2, b3]

In [3]:
def num_of_params(hs):
    num_bs = sum(hs[1:])
    num_wts = hs[0] * hs[1]
    for i in range(1,len(hs)-1):
        num_wts += hs[i] * hs[i+1]
    return(num_wts + num_bs)

num_of_params([3,3,3,3])

36

In [5]:
def g(x):
    return(1/(1+np.exp(-x)))

def g_hash(x):
    g_ = g(x)
    return(g_*(1-g_))

def O(x):
    return np.exp(x)/sum(np.exp(x))

def loss_cross_entropy(y, y_hat):
    return sum(y*(-np.log(y_hat)))

In [6]:
def forw_propagate(W,b,x):
    a = np.zeros((len(b),len(b),1))
    h = np.zeros((len(b),len(b),1))
    h[0] = x
    for i in range(len(b)):
        a[i] = np.dot(W[i],h[i-1] if i!=0 else x)+b[i]
        h[i] = g(a[i])
    y_hat = O(a[-1])
    return(a,h,y_hat)

a,h,y_hat = forw_propagate(W,b,x)

In [7]:
a[0].sum()

5.44766562448759

In [8]:
h[0].sum()

2.5749524957231924

In [9]:
[a[1].sum(),h[1].sum(),a[2].sum()], loss_cross_entropy(y, y_hat)

([6.460166773282593, 2.63139309587371, 4.874920988265704], array([0.85637856]))

In [10]:
-(y - y_hat)

array([[ 0.23691422],
       [ 0.33838847],
       [-0.57530268]])

In [11]:
def back_prop(W,b,y,y_hat,a,h):
    da = np.zeros((len(W),len(W),1))
    dW = np.zeros(((len(W),len(W),len(W))))
    db = np.zeros((len(W),len(W),1))
    dh = np.zeros((len(W),len(W),1))

    for i in range(len(W)-1,-1,-1):
        da[i] = -(y-y_hat) if i==len(W)-1 else dh[i]*g_hash(np.array(a[i]))
        dW[i] = da[i] * h[i-1].T
        db[i] = da[i]
        dh[i-1] = np.dot(W[i].T,da[i]) 
    print(db[1])
    W = W - dW
    b = b - db
    return(W, b)

W,b = back_prop(W,b,y,y_hat,a,h)
a,h,y_hat = forw_propagate(W,b,x)

[[ 0.01838198]
 [-0.01997644]
 [-0.0038401 ]]


In [12]:
loss_cross_entropy(y, y_hat)

array([0.07253369])